<a href="https://colab.research.google.com/github/Jhuliani/agente-gerador-de-rpg/blob/main/agente_criacao_de_rpg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework de agentes do Google
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
###############################################
# --- Agente 1
###############################################
def agente_cenario(tema: str) -> str:
    """Cria o cenário do RPG com base no tema fornecido"""
    cenario_agent = Agent(
        name="agente_cenario",
        model="gemini-1.5-pro",
        instruction=f"""
        Você é um mestre de RPG especializado em criação de mundos. Siga estas etapas:
        1. Analise o tema: '{tema}'
        2. Crie um cenário imersivo com:
           - Nome do mundo/região
           - Período histórico
           - Tom (sombrio, heroico, humorístico)
           - 2-3 conflitos principais
        3. Use 'Google Search' se precisar de referências históricas/mitológicas
        4. Retorne em formato narrativo (1 parágrafo) + lista de conflitos
        """,
        tools=[google_search],
        description="Cria cenários de RPG imersivos"
    )
    return call_agent(cenario_agent, tema)


In [ ]:
##############################################
# --- Agente 2
##############################################
def agente_npcs(cenario: str) -> str:
    """Gera NPCs interessantes para o cenário"""
    npc_agent = Agent(
        name="agente_npcs",
        model="gemini-1.5-pro",
        instruction=f"""
        Com base neste cenário: '{cenario}', crie 3 NPCs memoráveis:
        Para cada NPC inclua:
        - Nome, aparência e profissão
        - 1 motivação secreta
        - 1 fraqueza ou segredo
        - Como eles se relacionam com os conflitos do cenário
        """,
        description="Gera personagens não jogáveis com profundidade"
    )
    return call_agent(npc_agent, cenario)

In [ ]:
###########################################
# --- Agente 3
###########################################
def agente_missoes(cenario: str, npcs: str) -> str:
    """Cria missões coerentes com o cenário e NPCs"""
    missao_agent = Agent(
        name="agente_missoes",
        model="gemini-1.5-flash",
        instruction=f"""
        Com base neste cenário: '{cenario}' e nestes NPCs: '{npcs}',
        crie 3 missões interconectadas:
        1. Missão principal (objetivo final)
        2. Missão secundária (para desenvolvimento)
        3. Missão de intriga (relacionada a segredos)
        Para cada missão inclua:
        - Título e objetivo
        - NPCs envolvidos
        - Recompensas
        - 2-3 desafios esperados
        """,
        description="Cria quests para campanhas de RPG"
    )
    return call_agent(missao_agent, f"Cenário: {cenario}\nNPCs: {npcs}")

In [ ]:
#########################################
# --- Agente 4
#########################################
def agente_item_magico(cenario: str) -> str:
    """Gera itens mágicos temáticos"""
    item_agent = Agent(
        name="agente_itens",
        model="gemini-1.0-pro",
        instruction=f"""
        Para o cenário: '{cenario}', crie 2 itens mágicos únicos:
        Para cada item inclua:
        - Nome criativo
        - Descrição física
        - Efeito mágico/benefício
        - 1 efeito colateral ou maldição (se aplicável)
        - Valor estimado em peças de ouro
        """,
        description="Cria itens mágicos balanceados"
    )
    return call_agent(item_agent, cenario)

In [ ]:
###############################################
# --- Execução Principal --- #
###############################################

print("Bem-vindo ao seu Assistente para Criação de RPGs")
print("Vamos criar uma campanha incrível juntos!\n")

# Passo 1: Obter o tema do usuário
tema_rpg = input("Qual o tema da sua campanha (ex: 'fantasia sombria', 'cyberpunk noir', 'terror lovecraftiano')? ").strip()

print(f"\n Vamos criar uma campanha de {tema_rpg}!")

# Agente 1: Criar Cenário
cenario = agente_cenario(tema_rpg)
print("\n Resultado do Agente 1 (Cenário) \n")
display(to_markdown(cenario))
print("--------------------------------------------------------------")

# Agente 2: Criar NPCs
npcs = agente_npcs(cenario)
print("\n Resultado do Agente 2 (NPCs) \n")
display(to_markdown(npcs))
print("--------------------------------------------------------------")

# Agente 3: Criar Missões
missoes = agente_missoes(cenario, npcs)
print("\n Resultado do Agente 3 (Missões) \n")
display(to_markdown(missoes))
print("--------------------------------------------------------------")

# Agente 4: Criar Itens Mágicos
itens = agente_item_magico(cenario)
print("\n Resultado do Agente 4 (Itens Mágicos) \n")
display(to_markdown(itens))
print("--------------------------------------------------------------")

print("\n Campanha concluída! Agora é só reunir seus jogadores e começar a aventura!")